This is an example of a simple CNN developed, trained and utilized

AI was used to help generate the codebase

Note: Make sure that the tensorflow package is installed in your device.

In [1]:
# Lib imports
import os
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras import layers, models
import numpy as np

In [2]:
# DATASET DIRECTORY CONFIGURATION
# Download and unzip the dataset from Kaggle, set the directory paths accordingly.
train_dir = "D:/CODE/25-26/CS 3B/CHRISTIAN_PAULO_PILLADO/Exercise_6/dataset/train"  # e.g. './muffin-vs-chihuahua/train'
test_dir = "D:/CODE/25-26/CS 3B/CHRISTIAN_PAULO_PILLADO/Exercise_6/dataset/test"    # e.g. './muffin-vs-chihuahua/test'

In [3]:
# IMAGE PARAMETERS
# Used to resize the input images, also will determine the input size of your input layer.
IMG_SIZE = (128, 128)
BATCH_SIZE = 32

In [4]:
# DATA PREPROCESSING & AUGMENTATION
# Optional but recommended for image processing tasks, especially with limited data.
train_datagen = ImageDataGenerator(
    rescale=1./255,
    rotation_range=15,
    width_shift_range=0.1,
    height_shift_range=0.1,
    horizontal_flip=True,
    validation_split=0.2
)
test_datagen = ImageDataGenerator(rescale=1./255)

train_generator = train_datagen.flow_from_directory(
    train_dir,
    target_size=IMG_SIZE,
    batch_size=BATCH_SIZE,
    class_mode='binary',
    subset='training'
)
val_generator = train_datagen.flow_from_directory(
    train_dir,
    target_size=IMG_SIZE,
    batch_size=BATCH_SIZE,
    class_mode='binary',
    subset='validation'
)
test_generator = test_datagen.flow_from_directory(
    test_dir,
    target_size=IMG_SIZE,
    batch_size=BATCH_SIZE,
    class_mode='binary',
    shuffle=False
)

Found 575 images belonging to 2 classes.
Found 143 images belonging to 2 classes.
Found 106 images belonging to 2 classes.


Improvement of CNN Architecture

Improved Model Architecture with Regularization and Dropout

In [5]:
# IMPROVED CNN MODEL ARCHITECTURE with Regularization and Dropout
from tensorflow.keras import regularizers

# Lower learning rate for more stable training
initial_learning_rate = 0.0001
lr_schedule = tf.keras.optimizers.schedules.ExponentialDecay(
    initial_learning_rate,
    decay_steps=1000,
    decay_rate=0.96,
    staircase=True
)

optimizer = tf.keras.optimizers.Adam(learning_rate=lr_schedule)

# Model with L2 regularization and dropout
model_improved = models.Sequential([
    layers.Conv2D(32, (3, 3), activation='relu', 
                  kernel_regularizer=regularizers.l2(0.001),
                  input_shape=(IMG_SIZE[0], IMG_SIZE[1], 3)),
    layers.MaxPooling2D(2, 2),
    layers.Dropout(0.25),
    
    layers.Conv2D(64, (3, 3), activation='relu',
                  kernel_regularizer=regularizers.l2(0.001)),
    layers.MaxPooling2D(2, 2),
    layers.Dropout(0.25),
    
    layers.Conv2D(128, (3, 3), activation='relu',
                  kernel_regularizer=regularizers.l2(0.001)),
    layers.MaxPooling2D(2, 2),
    layers.Dropout(0.25),
    
    layers.Flatten(),
    layers.Dense(128, activation='relu',
                 kernel_regularizer=regularizers.l2(0.001)),
    layers.Dropout(0.5),
    layers.Dense(1, activation='sigmoid')
])

model_improved.compile(optimizer=optimizer, 
                      loss='binary_crossentropy', 
                      metrics=['accuracy'])
model_improved.summary()

c:\Users\cpill\AppData\Local\Programs\Python\Python313\Lib\site-packages\keras\src\layers\convolutional\base_conv.py:113: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ conv2d (Conv2D)                 │ (None, 126, 126, 32)   │           896 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d (MaxPooling2D)    │ (None, 63, 63, 32)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout (Dropout)               │ (None, 63, 63, 32)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_1 (Conv2D)               │ (None, 61, 61, 64)     │        18,496 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_1 (MaxPooling2D)  │ (None, 30, 30, 64)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_1 (Dropout)             │ (None, 30, 30, 64)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_2 (Conv2D)               │ (None, 28, 28, 128)    │        73,856 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_2 (MaxPooling2D)  │ (None, 14, 14, 128)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_2 (Dropout)             │ (None, 14, 14, 128)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten (Flatten)               │ (None, 25088)          │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (None, 128)            │     3,211,392 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_3 (Dropout)             │ (None, 128)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 1)              │           129 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 3,304,769 (12.61 MB)

 Trainable params: 3,304,769 (12.61 MB)

 Non-trainable params: 0 (0.00 B)

In [6]:
# TRAINING THE IMPROVED CNN
history_improved = model_improved.fit(
    train_generator,
    epochs=10, 
    validation_data=val_generator
)

Epoch 1/10
18/18 ━━━━━━━━━━━━━━━━━━━━ 47s 2s/step - accuracy: 0.5183 - loss: 1.0827 - val_accuracy: 0.5455 - val_loss: 1.0557
Epoch 2/10
18/18 ━━━━━━━━━━━━━━━━━━━━ 18s 1s/step - accuracy: 0.5774 - loss: 1.0349 - val_accuracy: 0.5594 - val_loss: 1.0243
Epoch 3/10
18/18 ━━━━━━━━━━━━━━━━━━━━ 23s 1s/step - accuracy: 0.6939 - loss: 0.9667 - val_accuracy: 0.7413 - val_loss: 0.9491
Epoch 4/10
18/18 ━━━━━━━━━━━━━━━━━━━━ 22s 1s/step - accuracy: 0.7391 - loss: 0.8745 - val_accuracy: 0.8042 - val_loss: 0.8760
Epoch 5/10
18/18 ━━━━━━━━━━━━━━━━━━━━ 23s 1s/step - accuracy: 0.7252 - loss: 0.8712 - val_accuracy: 0.8252 - val_loss: 0.8613
Epoch 6/10
18/18 ━━━━━━━━━━━━━━━━━━━━ 23s 1s/step - accuracy: 0.7739 - loss: 0.8296 - val_accuracy: 0.8392 - val_loss: 0.8384
Epoch 7/10
18/18 ━━━━━━━━━━━━━━━━━━━━ 20s 1s/step - accuracy: 0.7791 - loss: 0.8036 - val_accuracy: 0.8322 - val_loss: 0.8111
Epoch 8/10
18/18 ━━━━━━━━━━━━━━━━━━━━ 15s 837ms/step - accuracy: 0.7774 - loss: 0.7773 - val_accuracy: 0.8322 - val_lo

In [7]:
# EVALUATE THE IMPROVED MODEL
test_loss_improved, test_acc_improved = model_improved.evaluate(test_generator)
print(f"\n=== IMPROVED MODEL RESULTS ===")
print(f"Test Accuracy: {test_acc_improved:.4f}")
print(f"Test Loss: {test_loss_improved:.4f}")

# Compare with original if you have it
print(f"\n=== COMPARISON ===")
print(f"Improved Test Accuracy: {test_acc_improved:.4f}")

4/4 ━━━━━━━━━━━━━━━━━━━━ 2s 497ms/step - accuracy: 0.8019 - loss: 0.7671

=== IMPROVED MODEL RESULTS ===
Test Accuracy: 0.8019
Test Loss: 0.7671

=== COMPARISON ===
Improved Test Accuracy: 0.8019


In [8]:
# SAVE THE IMPROVED MODEL
model_improved.save('exercise_6_custom_PILLADO.h5')
print("Improved model saved as 'exercise_6_custom_PILLADO.h5'")

Improved model saved as 'exercise_6_custom_PILLADO.h5'


In [15]:
# SIMPLE INFERENCE SCRIPT
from tensorflow.keras.preprocessing import image

def predict_image(img_path, model_path='exercise_6_custom_PILLADO.h5'):
    model = tf.keras.models.load_model(model_path)
    img = image.load_img(img_path, target_size=IMG_SIZE)
    img_array = image.img_to_array(img) / 255.0
    img_array = np.expand_dims(img_array, axis=0)
    pred = model.predict(img_array)[0,0]
    label = "pizza" if pred >= 0.5 else "icecream"
    print(f"Prediction: {label} (confidence: {pred:.2f})")


In [16]:
# IMPROVED INFERENCE SCRIPT
def predict_image_improved(img_path, model):
    img = image.load_img(img_path, target_size=IMG_SIZE)
    img_array = image.img_to_array(img) / 255.0
    img_array = np.expand_dims(img_array, axis=0)
    pred = model.predict(img_array, verbose=0)[0,0]
    label = "pizza" if pred >= 0.5 else "icecream"
    confidence = pred if pred >= 0.5 else (1 - pred)
    return label, confidence, pred

# RUN 1: Predictions with improved model
print("=== RUN 1: IMPROVED MODEL PREDICTIONS ===")
test_images = [
    "D:/CODE/25-26/CS 3B/CHRISTIAN_PAULO_PILLADO/Exercise_6/predict/img5.jpg",
    "D:/CODE/25-26/CS 3B/CHRISTIAN_PAULO_PILLADO/Exercise_6/predict/img6.jpg",
    "D:/CODE/25-26/CS 3B/CHRISTIAN_PAULO_PILLADO/Exercise_6/predict/img7.jpg",
    "D:/CODE/25-26/CS 3B/CHRISTIAN_PAULO_PILLADO/Exercise_6/predict/img8.jpg",
]

for i, img_path in enumerate(test_images, 1):
    label, confidence, raw_pred = predict_image_improved(img_path, model_improved)
    print(f"Image {i}: {label} (Confidence: {confidence:.2%}, Raw: {raw_pred:.4f})")

# RUN 2: Second run with same model
print("\n=== RUN 2: IMPROVED MODEL PREDICTIONS (Second Run) ===")
for i, img_path in enumerate(test_images, 1):
    label, confidence, raw_pred = predict_image_improved(img_path, model_improved)
    print(f"Image {i}: {label} (Confidence: {confidence:.2%}, Raw: {raw_pred:.4f})")

=== RUN 1: IMPROVED MODEL PREDICTIONS ===
Image 1: pizza (Confidence: 70.86%, Raw: 0.7086)
Image 2: icecream (Confidence: 52.67%, Raw: 0.4733)
Image 3: pizza (Confidence: 79.92%, Raw: 0.7992)
Image 4: icecream (Confidence: 64.54%, Raw: 0.3546)

=== RUN 2: IMPROVED MODEL PREDICTIONS (Second Run) ===
Image 1: pizza (Confidence: 70.86%, Raw: 0.7086)
Image 2: icecream (Confidence: 52.67%, Raw: 0.4733)
Image 3: pizza (Confidence: 79.92%, Raw: 0.7992)
Image 4: icecream (Confidence: 64.54%, Raw: 0.3546)
